In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
# Add parent dir to path, so that python finds the lenskit package
sys.path.insert(0,parentdir)

In [2]:
from lenskit.metrics import dataGenerator
from lenskit.metrics import topnFair
from lenskit import batch, topn, util, topnFair
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als, user_knn as knn

In [3]:
import numpy as np
import pandas as pd
import math
%matplotlib inline

In [4]:
index=range(0,20)
#col_names1 = ["item", 'user']
col_names =  ['item', 'user', 'rank', 'protected']
my_df  = pd.DataFrame(index = index, columns = col_names)

my_df["item"] = np.arange(20)
my_df["user"] = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
my_df["rank"] = np.arange(20)
my_df["protected"] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1]


print(my_df)

                         
#data = np.array([np.arange(10)], [np.arange(10)])
#data.head

    item  user  rank  protected
0      0     1     0          0
1      1     1     1          0
2      2     1     2          0
3      3     1     3          0
4      4     1     4          0
5      5     1     5          0
6      6     1     6          0
7      7     1     7          0
8      8     1     8          0
9      9     1     9          0
10    10     1    10          0
11    11     1    11          0
12    12     1    12          0
13    13     1    13          0
14    14     1    14          0
15    15     1    15          0
16    16     1    16          0
17    17     1    17          0
18    18     1    18          1
19    19     1    19          1


In [5]:
res1= my_df.item.nunique()
print(res1)
res2 =  my_df.loc[my_df['protected'] == 1]
print(res2.item.nunique())


rla = topnFair.FairRecListAnalysis(['user'])
rla.add_metric("rND")
rla.add_metric("rKL")
rla.add_metric("rRD")
results = rla.compute(my_df, my_df, 'protected', items_N=res1, proItems_N=res2.item.nunique())
results.head()

20
2


,rND,rKL,rRD
user,,,
1,0.028906,0.041557,0.032118


In [6]:
#test rND for my_df = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1] 0/10 vs 2/20 
dif = abs((0/10)-(2/20)) 
rnd = (dif/math.log(10+1,2)+(0/math.log(20+1,2))) 
rnd       


0.02890648263178878

In [7]:
#test rND for my_df = [0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1] 1/10 vs 4/20 
dif1 = abs((1/10)-(4/20))

rnd1 = (dif1/math.log(10+1,2)+(0/math.log(20+1,2))) 
rnd1       

0.02890648263178878

In [8]:
#test rRD for my_df = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1] 0/10 vs 2/20 

ratio1 = 0/10
ratio2 = 2/18
dif = abs(ratio1-ratio2)
rrd = (dif/math.log(10+1,2)+(0/math.log(20+1,2))) 
rrd       

0.032118314035320866

In [9]:
print(topnFair.getNormalizer(20,2,"rRD"))
nrRD = rrd/topnFair.getNormalizer(20,2,"rRD")
nrRD


0.03211831403532086


1.0000000000000002

In [10]:
#test rRD for my_df = [0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,1] 2/10 vs 4/20 

ratio1 =  2/8
ratio2 = 4/16
dif = abs(ratio1-ratio2)
rrd = (dif/math.log(10+1,2)+(0/math.log(20+1,2))) 
rrd       

0.0

In [11]:
#test rRD for my_df = [0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1] 1/10 vs 4/20 

ratio1 = 1/9
ratio2 = 4/16
dif = abs(ratio1-ratio2)
rrd = (dif/math.log(10+1,2)+(0/math.log(20+1,2))) 
rrd       

0.04014789254415109

In [17]:
#test rRD for my_df = 5/10 vs 10/20 

ratio1 = 5/5
ratio2 = 10/10
dif = abs(ratio1-ratio2)
rrd1 = (dif/math.log(10+1,2)+(0/math.log(20+1,2))) 
rrd1       

0.0

In [18]:
print(topnFair.getNormalizer(20,10,"rRD"))
nrRD = rrd1/topnFair.getNormalizer(20,10,"rRD")
nrRD

0.28906482631788777


0.0

In [12]:
#test rKL for my_df["protected"] = [0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1] 1/10 vs 4/20

px= 1/10
qx=4/20

rkl =(px*math.log(px/qx,2))+(1-px)*math.log((1-px)/(1-qx),2)

print(rkl)
print(rkl/math.log(11,2))

0.052932501298081136
0.015300924294301195


In [13]:
#test rKL for my_df["protected"] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1] 0/10 vs 2/20
# corner case -- px cant be 0 thus set manuale to 0,001

#px= 0/10 - px cant be 0 thus set manuale to 0,001
px = 0.001
qx=2/20

rkl2 =(px*math.log(px/qx,2))+(1-px)*math.log((1-px)/(1-qx),2)

print(rkl2)
print(rkl2/math.log(11,2))

0.143765260709031
0.04155748011740191
